# Testing the web camerA

In [ ]:
import cv2  # OpenCV library for webcam and image processing

# Open webcam (index 0 is default camera—use 1 or 2 if multiple cameras)
cap = cv2.VideoCapture(0)

# Check if webcam opened
if not cap.isOpened():
    print("Error: Could not open webcam.")  # Error if webcam isn’t detected
    exit()  # Exit script if no webcam

# Loop to show live feed
while True:
    ret, frame = cap.read()  # Read frame: ret is True if successful, frame is image array
    if not ret:
        print("Error: Could not read frame.")  # Error if frame capture fails
        break  # Exit loop on failure
    
    cv2.imshow("Webcam Test", frame)  # Display frame in a window named "Webcam Test"
    
    # Wait 1ms for keypress, check for 'q' (ASCII 113) to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Exit loop on 'q'

# Cleanup
cap.release()  # Release webcam resource
cv2.destroyAllWindows()  # Close all OpenCV windows

## Add Hand Detection with Mediapipe

In [3]:
import cv2  # For webcam and image processing
import mediapipe as mp  # For hand detection and landmarks

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands  # Load hands detection module
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)  # Detect 1 hand, 70% confidence threshold
mp_draw = mp.solutions.drawing_utils  # Tool to draw hand landmarks

# Open webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

print("Webcam opened successfully! Press 'q' to quit.")  # User feedback

# Loop for live hand detection
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break
    
    # Convert frame from BGR (OpenCV) to RGB (Mediapipe needs RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process frame to detect hands
    results = hands.process(rgb_frame)  # Returns hand landmarks or None if no hand
    
    # Draw landmarks if hand detected
    if results.multi_hand_landmarks:  # Check if any hands found
        for hand_landmarks in results.multi_hand_landmarks:  # Loop through detected hands (1 here)
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)  # Draw green dots and lines
    
    cv2.imshow("Hand Detection", frame)  # Show frame with landmarks
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
hands.close()  # Close Mediapipe Hands instance

Webcam opened successfully! Press 'q' to quit.


## collecting data

In [ ]:
import cv2  # For webcam and saving images
import mediapipe as mp  # For hand detection
import os  # For creating directories

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Define new gestures—your choice (e.g., What's up: palm up, Not good: thumbs-down, Good: OK sign)
gestures = ["whats_up", "not_good", "good"]
for gesture in gestures:
    os.makedirs(f"dataset/{gesture}", exist_ok=True)  # Create folder for each gesture—overwrites old data

# Open webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

print("Press 'w' for What's up, 'n' for Not good, 'g' for Good, 'q' to quit.")  # Key instructions

# Variables for collection
current_gesture = None  # Tracks which gesture we’re collecting
count = 0  # Counts images per gesture

# Loop to collect data
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break
    
    frame = cv2.flip(frame, 1)  # Mirror feed—makes it intuitive (right hand on right), matches live prediction
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    # Show landmarks for feedback
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    # Display current collection status
    if current_gesture:
        cv2.putText(frame, f"Collecting: {current_gesture} ({count})", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow("Gesture Collection", frame)
    
    # Handle key presses
    key = cv2.waitKey(1) & 0xFF
    if key == ord('w'):  # Start collecting "What's up"
        current_gesture = "whats_up"
        count = 0
    elif key == ord('n'):  # Start "Not good"
        current_gesture = "not_good"
        count = 0
    elif key == ord('g'):  # Start "Good"
        current_gesture = "good"
        count = 0
    elif key == ord('q'):  # Quit
        break
    
    # Save image if collecting and hand detected
    if current_gesture and results.multi_hand_landmarks and count < 400:
        filename = f"dataset/{current_gesture}/{count}.jpg"  # e.g., dataset/whats_up/0.jpg
        cv2.imwrite(filename, frame)  # Save frame as JPEG
        count += 1  # Increment count
        print(f"Saved {filename}")  # Confirm save
    
    # Stop collecting after 400
    if count >= 400:
        current_gesture = None  # Move to next gesture

cap.release()
cv2.destroyAllWindows()
hands.close()

## preprocessing the data ,making redy to feed in model

In [ ]:
import cv2  # For loading and processing images
import os  # For file operations
import numpy as np  # For array conversion
from sklearn.model_selection import train_test_split  # For splitting data

# Define parameters
IMG_SIZE = (224, 224)  # VGG16 expects 224x224 RGB images
gestures = ["whats_up", "not_good", "good"]  # New gestures
data_dir = "dataset"  # Root folder with images

# Lists to store data
data = []  # Holds preprocessed images
labels = []  # Holds labels (0, 1, 2)

# Load and preprocess images
for gesture in gestures:
    gesture_path = os.path.join(data_dir, gesture)  # e.g., dataset/whats_up
    label = gestures.index(gesture)  # 0 = whats_up, 1 = not_good, 2 = good
    
    for img_file in os.listdir(gesture_path):  # Loop through images (e.g., 0.jpg)
        img_path = os.path.join(gesture_path, img_file)  # Full path
        img = cv2.imread(img_path)  # Load image in BGR format
        if img is None:
            print(f"Failed to load {img_path}")  # Warn if image fails
            continue
        
        img = cv2.resize(img, IMG_SIZE)  # Resize to 224x224—VGG16 input size
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB—VGG16 expects RGB
        img = img / 255.0  # Normalize pixels from 0-255 to 0-1—matches ImageNet pretraining
        
        data.append(img)  # Add image to list
        labels.append(label)  # Add corresponding label

# Convert lists to NumPy arrays
data = np.array(data, dtype="float32")  # e.g., (1200, 224, 224, 3)—float32 for TensorFlow
labels = np.array(labels)  # e.g., (1200,)—integer labels

# Split into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42  # 20% test, fixed seed for reproducibility
)

# Print shapes to verify
print("Training data shape:", X_train.shape)  # e.g., (960, 224, 224, 3)
print("Testing data shape:", X_test.shape)    # e.g., (240, 224, 224, 3)
print("Training labels shape:", y_train.shape)  # e.g., (960,)
print("Testing labels shape:", y_test.shape)    # e.g., (240,)

# Save preprocessed data
np.save("X_train.npy", X_train)  # Save training images
np.save("X_test.npy", X_test)    # Save testing images
np.save("y_train.npy", y_train)  # Save training labels
np.save("y_test.npy", y_test)    # Save testing labels